<H1> Introdução </H1>
<p> 
    Os dados analisados aqui estão disponíveis no site do Instituto Nacional de Metereologia (INMET) em formato CSV e podem ser baixados através <a href="https://portal.inmet.gov.br/dadoshistoricos"> deste link </a> e por isso não foram incluídos no repositório. 
</p>

<p> 
    Os dados foram mantidos na organização original, com as tabelas divididas em pastas por ano e em arquivos segundo o local de medição. 
</p>

<p> Uma lista de todos os arquivos disponíveis foi salva no arquivo <i> files.txt </i> para facilitar a consulta dos mesmos e para permitir a quem esteja acessando este repositório consultar rapidamente quais arquivos estão disponíveis sem necessidade de baixá-los no site do INMET. </p>

<p> 
Os dados de 2024 não foram incluídos pois no momento em que este repositório foi criado o ano ainda não tinha acabado. Estes dados serão incluídos em algum momento no futuro. 
</p>

In [3]:
# Bibliotecas utilizadas
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt



In [4]:
# Lê a lista de arquivos e criar um dataframe com os dados. 
file = open("files.txt", "r")
file_content = file.read()
file_list = [x for x in file_content.splitlines() if len(x) > 2]
file_list_splitted = [ x.replace(".CSV","").strip().split('_')[0 : 8] + [x] for x in file_list]
df_files = pd.DataFrame(columns=["INMET", "REGIAO", "ESTADO", "CODIGO", "CIDADE", "DATA INI", "A", "DATA FIM", "ARQUIVO"],
                        data = file_list_splitted).drop(columns={"INMET", "A"})